# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 61 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.13618


extracting tarball to tmp_2209.13618...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.13651


extracting tarball to tmp_2209.13651...

...

/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.14159/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.14166...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.14226


extracting tarball to tmp_2209.14226...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13717-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13717) | **Polarized Emission From Four Supernova Remnants In The THOR Survey**  |
|| Russell Shanahan, et al. -- incl., <mark>Henrik Beuther</mark>, <mark>Jonas Syed</mark> |
|*Appeared on*| *2022-09-29*|
|*Comments*| *Accepted for publication in the Astrophysical Journal. Figures 6, 19 and 20 may not be displayed in the browser pdf viewer, but the downloaded pdf is complete*|
|**Abstract**| We present polarization and Faraday rotation for the supernova remnants (SNRs) G46.8-0.3, G43.3-0.2, G41.1-0.3, and G39.2-0.3 in L-band (1-2 GHz) radio continuum in The HI/OH/Recombination line (THOR) survey. We detect polarization from G46.8-0.3, G43.3-0.2 and G39.2-0.3 but find upper limits at the 1% level of Stokes I for G41.1-0.3. For G46.8-0.3 and G39.2-0.3 the fractional polarization varies on small scales from 1% to ~6%. G43.3-0.2 is less polarized with fractional polarization <~3%. We find upper limits at the 1% level for the brighter regions in each SNR with no evidence for associated enhanced Faraday depolarization. We observe significant variation in Faraday depth and fractional polarization on angular scales down to the resolution limit of 16". Approximately 6% of our polarization detections from G46.8-0.3 and G39.2-0.3 exhibit two-component Faraday rotation and 14% of polarization detections in G43.3-0.2 are multi-component. For G39.2-0.3 we find a bimodal Faraday depth distribution with a narrow peak and a broad peak for all polarization detections as well as for the subset with two-component Faraday rotation. We identify the narrow peak with the front side of the SNR and the broad peak with the back side. Similarly, we interpret the observed Faraday depth distribution of G46.8-0.3 as a superposition of the distributions from the front side and the back side. We interpret our results as evidence for a partially filled shell with small-scale magnetic field structure and internal Faraday rotation. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13618-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13618) | **The need for multicomponent dust attenuation in modeling nebular  emission: constraints from SDSS-IV MaNGA**  |
|| Xihan Ji, et al. -- incl., <mark>Cheng Li</mark> |
|*Appeared on*| *2022-09-29*|
|*Comments*| *26 pages, 17 figures, submitted to A&A*|
|**Abstract**| A fundamental assumption adopted in nearly every extragalactic emission-line study is that the attenuation of different emission lines can be described by a single attenuation curve. Here we show this assumption fails in many cases with important implications for derived results. We developed a new method to measure the differential nebular attenuation among three kinds of transitions: the Balmer lines of hydrogen, high-ionization transitions, and low-ionization transitions. This method bins the observed data in a multidimensional space spanned by attenuation-insensitive line ratios. Within each small bin, the variations in line ratios are mainly driven by the variations in the nebular attenuation. This allows us to measure the nebular attenuation using both forbidden lines and Balmer lines. We applied this method to a sample of 2.4 million H II region spaxels from SDSS-IV MaNGA. We found that the attenuation of high ionization lines and Balmer lines can be well described by a single Fitzpatrick (1999) extinction curve with $R_V=3.1$. However, no single attenuation curve can simultaneously account for all three transitions. This strongly suggests that different lines have different effective attenuations, likely because spectroscopy at kilo-parsecs resolution mixes multiple regions with different intrinsic line ratios and different levels of attenuation. As a result, the assumption that different lines follow the same attenuation curve breaks down. Using a single attenuation curve determined by Balmer lines to correct attenuation-sensitive forbidden line ratios could bias the nebular parameters derived by 0.06--0.25 dex at $A_V = 1$, depending on the details of the dust attenuation model. Observations of a statistically large sample of H II regions with high spatial resolutions and large spectral coverage are vital for improved modeling and deriving accurate corrections for this effect. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14166-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14166) | **Colour gradients of low-redshift galaxies in the DESI Legacy Imaging  Survey**  |
|| <mark>Li-Wen Liao</mark>, Andrew Cooper |
|*Appeared on*| *2022-09-29*|
|*Comments*| *26 pages, 26 figures, Submitted to MNRAS*|
|**Abstract**| Radial colour gradients within galaxies arise from gradients of stellar age, metallicity and dust reddening. Large samples of colour gradients from wide-area imaging surveys can complement smaller integral-field spectroscopy datasets and can be used to constrain galaxy formation models. Here we measure colour gradients for low-redshift galaxies (z<0.1) using photometry from the DESI Legacy Imaging Survey DR9. Our sample comprises ~93,000 galaxies with spectroscopic redshifts and ~574,000 galaxies with photometric redshifts. We focus on gradients across a radial range 0.5 Re to Re, which corresponds to the inner disk of typical late type systems at low redshift. This region has been the focus of previous statistical studies of colour gradients and has recently been explored by spectroscopic surveys such as MaNGA. We find the colour gradients of most galaxies in our sample are negative (redder towards the centre), consistent with the literature. We investigate empirical relationships between colour gradient, average $g-r$ and $r-z$ colour, $M_r$, $M_\star$, and sSFR. Trends of gradient strength with $M_r$ ($M_\star$) show an inflection around $M_r\sim-21$ ($\log_{10} \, M_\star/\mathrm{M_\odot}\sim10.5$). Below this mass, colour gradients become steeper with increasing $M_\star$, whereas colour gradients in more massive galaxies become shallower. We find that positive gradients (bluer stars at smaller radii) are typical for galaxies of $M_{\star}\sim10^{8}\,\mathrm{M_\odot}$. We compare our results to age and metallicity gradients in two datasets derived from fits of different stellar population libraries to MaNGA spectra, but find no clear consensus explanation for the trends we observe. Both MaNGA datasets seem to imply a significant contribution from dust reddening, in particular, to explain the flatness of colour gradients along the red sequence. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14113-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14113) | **Improving ANAIS-112 sensitivity to DAMA/LIBRA signal with machine  learning techniques**  |
|| I. Coarasa, et al. -- incl., <mark>P. Villar</mark> |
|*Appeared on*| *2022-09-29*|
|*Comments*| **|
|**Abstract**| The DAMA/LIBRA observation of an annual modulation in the detection rate compatible with that expected for dark matter particles from the galactic halo has accumulated evidence for more than twenty years. It is the only hint of a direct detection of the elusive dark matter, but it is in strong tension with the negative results of other very sensitive experiments, requiring ad-hoc scenarios to reconcile all the present experimental results. Testing the DAMA/LIBRA result using the same target material, NaI(Tl), removes the dependence on the particle and halo models and is the goal of the ANAIS-112 experiment, taking data at the Canfranc Underground Laboratory in Spain since August 2017 with 112.5 kg of NaI(Tl). At very low energies, the detection rate is dominated by non-bulk scintillation events and careful event selection is mandatory. This article summarizes the efforts devoted to better characterize and filter this contribution in ANAIS-112 data using a boosted decision tree (BDT), trained for this goal with high efficiency. We report on the selection of the training populations, the procedure to determine the optimal cut on the BDT parameter, the estimate of the efficiencies for the selection of bulk scintillation in the region of interest (ROI), and the evaluation of the performance of this analysis with respect to the previous filtering. The improvement achieved in background rejection in the ROI, but moreover, the increase in detection efficiency, push the ANAIS-112 sensitivity to test the DAMA/LIBRA annual modulation result beyond 3$\sigma$ with three-year exposure, being possible to reach 5$\sigma$ by extending the data taking for a few more years than the scheduled 5 years which were due in August 2022. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14159-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14159) | **Spiral Arms are Metal Freeways: Azimuthal Gas-Phase Metallicity  Variations in Simulated Cosmological Zoom-in Flocculent Disks**  |
|| Matthew E. Orr, et al. -- incl., <mark>Andrew Wetzel</mark> |
|*Appeared on*| *2022-09-29*|
|*Comments*| *16 Pages, 14 Figures, submitted to MNRAS*|
|**Abstract**| We examine the azimuthal variations in gas-phase metallicity profiles in simulated Milky Way mass disk galaxies from the Feedback in Realistic Environments (FIRE-2) cosmological zoom-in simulation suite, which includes a sub-grid turbulent metal mixing model. We produce spatially resolved maps of the disks at $z \approx 0$ with pixel sizes ranging from 250 to 750~pc, analogous to modern integral field unit (IFU) galaxy surveys, mapping the gas-phase metallicities in both the cold & dense gas and the ionized gas correlated with HII regions. We report that the spiral arms alternate in a pattern of metal rich and metal poor relative to the median metallicity on the order of $\lesssim 0.1$~dex, appearing generally in this sample of flocculent spirals. The pattern persists even in a simulation with different strengths of metal mixing, indicating that the pattern emerges from physics above the sub-grid scale. Local enrichment does not appear to be the dominant source of the azimuthal metallicity variations at $z \approx 0$: there is no correlation with local star formation on these spatial scales. Rather, the arms are moving inwards and outwards relative to each other, carrying their local metallicity gradients with them radially before mixing into the larger-scale interstellar medium. We propose that the arms act as freeways channeling relatively metal poor gas radially inwards, and relatively enriched gas radially outwards. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14226-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14226) | **Radio source-component association for the LOFAR Two-metre Sky Survey  with region-based convolutional neural networks**  |
|| Rafaël I.J. Mostert, et al. -- incl., <mark>Lara Alegre</mark>, <mark>Wendy L. Williams</mark> |
|*Appeared on*| *2022-09-29*|
|*Comments*| *22 pages; accepted for publication in A&A*|
|**Abstract**| Radio loud active galactic nuclei (RLAGNs) are often morphologically complex objects that can consist of multiple, spatially separated, components. Astronomers often rely on visual inspection to resolve radio component association. However, applying visual inspection to all the hundreds of thousands of well-resolved RLAGNs that appear in the images from the Low Frequency Array (LOFAR) Two-metre Sky Survey (LoTSS) at $144$ MHz, is a daunting, time-consuming process, even with extensive manpower. Using a machine learning approach, we aim to automate the radio component association of large ($> 15$ arcsec) radio components. We turned the association problem into a classification problem and trained an adapted Fast region-based convolutional neural network to mimic the expert annotations from the first LoTSS data release. We implemented a rotation data augmentation to reduce overfitting and simplify the component association by removing unresolved radio sources that are likely unrelated to the large and bright radio components that we consider using predictions from an existing gradient boosting classifier. For large ($> 15$ arcsec) and bright ($> 10$ mJy) radio components in the LoTSS first data release, our model provides the same associations for $85.3\%\pm0.6$ of the cases as those derived when astronomers perform the association manually. When the association is done through public crowd-sourced efforts, a result similar to that of our model is attained. Our method is able to efficiently carry out manual radio-component association for huge radio surveys and can serve as a basis for either automated radio morphology classification or automated optical host identification. This opens up an avenue to study the completeness and reliability of samples of radio sources with extended, complex morphologies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13982-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13982) | **Improved asteroseismic inversions for red-giant surface rotation rates**  |
|| F. Ahlborn, et al. -- incl., <mark>S. Hekker</mark> |
|*Appeared on*| *2022-09-29*|
|*Comments*| *17 pages, 22 figures, accepted for publication in Astronomy and Astrophysics*|
|**Abstract**| Asteroseismic observations of internal stellar rotation have indicated a substantial lack of angular momentum transport in theoretical models of subgiant and red-giant stars. Accurate core and surface rotation rate measurements are therefore needed to constrain internal transport processes included in the models. We eliminate substantial systematic errors of asteroseismic surface rotation rates found in previous studies. We propose a new objective function for the Optimally Localized Averages method of rotational inversions for red-giant stars, which results in more accurate envelope rotation rate estimates obtained from the same data. We use synthetic observations from stellar models across a range of evolutionary stages and masses to demonstrate the improvement. We find that our new inversion technique allows us to obtain estimates of the surface rotation rate that are independent of the core rotation. For a star at the base of the red-giant branch, we reduce the systematic error from about 20% to a value close to 0, assuming constant envelope rotation. We also show the equivalence between this method and the method of linearised rotational splittings. Our new rotational inversion method substantially reduces the systematic errors of red-giant surface rotation rates. In combination with independent measures of the surface rotation rate, this will allow better constraints to be set on the internal rotation profile. This will be a very important probe to further constrain the internal angular momentum transport along the lower part of the red-giant branch. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure .pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.13789-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.13789) | **The role and contribution of magnetic fields, characterized via their  magnetic flux, to the statistical structuring of the solar atmosphere**  |
|| <mark>K.J. Li</mark>, J.C. Xu, W. Feng |
|*Appeared on*| *2022-09-29*|
|*Comments*| *12 pages, 5 figures*|
|**Abstract**| The anomalous heating of the solar upper atmosphere is one of the eight key problems in modern astronomy. Moreover, the stratification of the solar atmosphere is an outstanding key-problem in solar physics. In this study, a hot butterfly-like pattern is found to run through the chromosphere to the corona lying right on top of the magnetic butterfly pattern of sunspots in the photosphere. We thus propose to introduce the term butterfly body to describe the butterfly diagram in the 3-dimensional atmosphere. Besides, we discuss the so-called polar brightening in different layers. It is found to be statistically in anti-phase with the solar cycle in the photosphere and the chromosphere, while in phase with the solar cycle in the corona. Accordingly, we describe the role and relationship of solar magnetic elements of different magnetic flux strengths to explain the statistical structuring of the solar atmosphere with the butterfly body over the solar cycle. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.13717.md
    + _build/html/tmp_2209.13717/./f6.png
    + _build/html/tmp_2209.13717/./f9.png
    + _build/html/tmp_2209.13717/./f3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Polarized Emission From Four Supernova Remnants In The THOR Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.13717-b31b1b.svg)](https://arxiv.org/abs/2209.13717)<mark>Appeared on: 2022-09-29</mark> - _Accepted for publication in the Astrophysical Journal. Figures 6, 19 and 20 may not be displayed in the browser pdf viewer, but the downloaded pdf is complete_

</div>
<div id="authors">

Russell Shanahan, et al. -- incl., <mark>Henrik Beuther</mark>, <mark>Jonas Syed</mark>

</div>
<div id="abstract">

**Abstract:** We present polarization and Faraday rotation for the supernova remnants (SNRs) G46.8$-$0.3, G43.3$-$0.2, G41.1$-$0.3, and G39.2$-$0.3 in L-band (1-2 GHz) radio continuum in The HI/OH/Recombination line (THOR) survey.  We detect polarization from G46.8$-$0.3, G43.3$-$0.2 and G39.2$-$0.3 but find upper limits at the 1\%level of Stokes$I$for G41.1$-$0.3.  For G46.8$-$0.3 and G39.2$-$0.3 the fractional polarization varies on small scales from 1\%to$\sim$6\%.  G43.3$-$0.2 is less polarized with fractional polarization$\lesssim$3\%.  We find upper limits at the 1\%level for the brighter regions in each SNR with no evidence for associated enhanced Faraday depolarization.  We observe significant variation in Faraday depth and fractional polarization on angular scales down to the resolution limit of 16$\arcsec$.  Approximately 6\%of our polarization detections from G46.8$-$0.3 and G39.2$-$0.3 exhibit two-component Faraday rotation and 14\%of polarization detections in G43.3$-$0.2 are multi-component.  For G39.2$-$0.3 we find a bimodal Faraday depth distribution with a narrow peak and a broad peak for all polarization detections as well as for the subset with two-component Faraday rotation.  We identify the narrow peak with the front side of the SNR and the broad peak with the back side.  Similarly, we interpret the observed Faraday depth distribution of G46.8$-$0.3 as a superposition of the distributions from the front side and the back side.  We interpret our results as evidence for a partially filled shell with small-scale magnetic field structure and internal Faraday rotation.

</div>

<div id="div_fig1">

<img src="tmp_2209.13717/./f6.png" alt="Fig8" width="100%"/>

**Figure 8. -** Grid plot of the Faraday depth spectra for the red box seen in Figure \ref{fig:G46.8-0.3_RM-Map}.  The horizontal limits are $-1500 \text{ rad m}^{-2} < \phi < 1500 \text{ rad m}^{-2}$ and vertical axis limits are $0 < P < 0.95$ mJy.  The grey region illustrates our Faraday depth rejection range.  The red horizontal line is our detection threshold where $P = 0.27$ mJy.  Colours indicate spectra where zero (black), one (blue) or two (purple) peaks satisfy our detection criteria (see Appendix \ref{appendix:DT}).  The coordinates of the bottom left and top right corners are $(\ell, b)=(46\fdg802, 0\fdg377)$ and $(\ell, b)=(46\fdg777, -0\fdg352)$.  The physical size of this region is $2.36$ pc on each side. (*fig:G46.8-0.3_RM_Grid2*)

</div>
<div id="div_fig2">

<img src="tmp_2209.13717/./f9.png" alt="Fig11" width="100%"/>

**Figure 11. -** Fractional polarization map of SNR G46.8$-$0.3.  Contours are from the THOR+VGPS map at 14, 19, 24, 29, 34 and 39 mJy/beam.  The coloured subregions were calculated by $\Pi = P/(I - I_{\text{bg}})$ where $\Pi$ is the fractional polarization, $P$ is the peak from the Faraday depth spectrum, $I$ is the Stokes $I$ from the THOR+VGPS map and $I_{\text{bg}}$ is the average background Stokes $I$.  The black subregions are upper limits for regions where polarized signal is heavily affected by leakage.  The criteria for these subregions is that $\Pi_{\text{upper}} = P_{\text{limit}}/(I - I_{\text{bg}}) < 1\%$, where $P_{\text{limit}}$ is the maximum from the noise profiles. (*fig:G46.8-0.3_PP-Map*)

</div>
<div id="div_fig3">

<img src="tmp_2209.13717/./f3.png" alt="Fig6" width="100%"/>

**Figure 6. -** Faraday depth map of SNR G46.8$-$0.3. The contour levels correspond to 14, 19, 24, 29, 34 and 39 mJy/beam. The subregions with a purple edge indicate the locations of two-component Faraday rotation.  The Faraday depth shown for subregions with two-component Faraday rotation is derived from the strongest peak.  The red box indicates the region where we present the individual Faraday depth spectra as a grid plot seen in Figure \ref{fig:G46.8-0.3_RM_Grid2}. (*fig:G46.8-0.3_RM-Map*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

48  publications in the last 7 days.
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers